# Eye-Tracking Data Analysis for PLR Tests
This notebook implements analysis of Pupillary Light Reflex (PLR) tests using eye-tracking data.
The analysis includes data cleaning, pupil size extraction, signal quality assessment, and biomarker calculation.

In [ ]:
# Import required libraries

# pandas - For data manipulation and analysis of tabular data (landmarks and protocol files)
import pandas as pd

# numpy - For numerical operations and array manipulation of pupil measurements
import numpy as np

# matplotlib - For creating static visualizations of pupil response data
import matplotlib.pyplot as plt

# pathlib - For cross-platform file path handling
from pathlib import Path

# logging - For tracking program execution and debugging information
import logging

# scipy.signal - For signal processing operations like Savitzky-Golay filtering
from scipy import signal

# seaborn - For enhanced statistical data visualization built on matplotlib
import seaborn as sns

# shapely.geometry - For polygon area calculation
from shapely.geometry import Polygon

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

## Data Validating Functions
Functions to validate landmarks and protocol files

In [ ]:
def load_landmarks_file(file_path):
    """
    Load and validate landmarks data file
    
    Args:
        file_path (str): Path to landmarks CSV file
        
    Returns:
        pd.DataFrame: Processed landmarks data
    """
    try:
        df = pd.read_csv(file_path)
        
        # Required columns for timestamp and status
        base_columns = ['timestamp', 'id', 'retcode']
        
        # Generate expected landmark column names for both eyes
        landmark_columns = []
        for eye in ['left', 'right']:
            for i in range(1, 28):  # 27 landmarks per eye
                landmark_columns.extend([
                    f'{eye}_lm_{i}_x',
                    f'{eye}_lm_{i}_y'
                ])
        
        required_columns = base_columns + landmark_columns
        
        # Verify all required columns exist
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            raise ValueError(f"Missing required columns: {', '.join(missing_columns)}")
            
        logger.info(f"Successfully validated landmarks file {file_path.name} with {len(df)} rows")
        return df
    except Exception as e:
        logger.error(f"Error loading landmarks file: {e}")
        raise

In [ ]:
def load_protocol_file(file_path):
    """
    Load and validate protocol data file
    
    Args:
        file_path (str): Path to protocol CSV file
        
    Returns:
        pd.DataFrame: Protocol timing data
    """
    try:
        df = pd.read_csv(file_path)
        required_columns = ['time', 'event']
        if not all(col in df.columns for col in required_columns):
            raise ValueError("Missing required columns in protocol file")
        return df
    except Exception as e:
        logger.error(f"Error loading protocol file: {e}")
        raise

In [ ]:
data_path = Path().cwd().parent / "data"
for subfolder in data_path.iterdir():
    if subfolder.is_dir():
        landmarks_file = subfolder / f'{subfolder.name}_plr_landmarks.csv'
        protocol_file = subfolder / f'{subfolder.name}_plr_protocol.csv'
        
        # Load landmarks and protocol files
        landmarks_df = load_landmarks_file(landmarks_file)
        protocol_df = load_protocol_file(protocol_file)

## Data Cleaning
Functions to clean and preprocess the landmarks data

In [ ]:
def clean_landmarks_data(df):
    """
    Clean landmarks data by removing invalid frames
    
    Args:
        df (pd.DataFrame): Raw landmarks data
        
    Returns:
        pd.DataFrame: Cleaned landmarks data
    """
    # Remove frames with invalid retCode
    cleaned_df = df[df['retcode'] == 'OK'].copy()
    
    # Reset index after filtering
    cleaned_df.reset_index(drop=True, inplace=True)
    
    logger.info(f"Removed {len(df) - len(cleaned_df)} frames with invalid retCode")
    return cleaned_df

In [ ]:
# Find the landmarks.csv file
landmarks_file = list(data_path.glob('**/*landmarks.csv'))[0]

# Load landmarks data into DataFrame
landmarks_df = pd.read_csv(landmarks_file)

# Display first few rows
landmarks_df.head()